# 使用LawFormer进行文本编码

In [1]:
from transformers import AutoModel, AutoTokenizer
import pandas as pd
import tqdm as tqdm
import numpy as np

/home/zhouyifan/anaconda3/envs/SearchMilvus/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at thunlp/Lawformer were not used when initializing LongformerModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model th

In [2]:
# inputs = tokenizer("任某提起诉讼，请求判令解除婚姻关系并对夫妻共同财产进行分割。", return_tensors="pt")
# # test demo
# inputs = tokenizer.encode("任某提起诉讼，请求判令解除婚姻关系并对夫妻共同财产进行分割。", return_tensors="pt")
tokenizer = AutoTokenizer.from_pretrained("thunlp/Lawformer")
model = AutoModel.from_pretrained("thunlp/Lawformer")

In [3]:
df = pd.read_csv('../../cleand_shanghai_min_hang_ren_min_fa_yuan.csv')
df.head()

,id,name,cause_of_action,type,court,party,date1,procuratorate,date2,doc,instance,num
0,1294988,张伟非法持有枪支一审刑事判决书,非法持有、私藏枪支、弹药,刑事案件,上海市闵行区人民法院,张伟,2017-09-19,上海市闵行区人民检察院,2016-04-25,上海市闵行区人民法院\n刑 事 判 决 书\n（2016）沪0112刑初647号\n公诉机关...,一审,（2016）沪0112刑初647号
1,1294989,深圳市赢时通汽车服务有限公司上海分公司与姜华车辆租赁合同纠纷一审民事裁定书,车辆租赁合同纠纷,民事案件,上海市闵行区人民法院,"深圳市赢时通汽车服务有限公司上海分公司,姜华",2018-01-30,NaN,2017-03-01,上海市闵行区人民法院\n民 事 裁 定 书\n（2017）沪0112民初2332号\n原告：...,一审,（2017）沪0112民初2332号
2,1294990,一汽汽车金融有限公司与王以新金融借款合同纠纷一审民事判决书,金融借款合同纠纷,民事案件,上海市闵行区人民法院,"汽汽车金融有限公司,王以新",2018-04-01,NaN,2017-11-30,上海市闵行区人民法院\n民 事 判 决 书\n（2017）沪0112民初15632号\n原告...,一审,（2017）沪0112民初15632号
3,1294993,上海宝隆物业管理有限公司物业服务合同纠纷一审民事裁定书,物业服务合同纠纷,民事案件,上海市闵行区人民法院,"上海宝隆物业管理有限公司,樊美芬",2017-11-03,NaN,2017-10-20,上海市闵行区人民法院\n民 事 裁 定 书\n（2017）沪0112民初33632号\n原告...,一审,（2017）沪0112民初33632号
4,1294994,汪秀维、毛明华房屋租赁合同纠纷一审民事裁定书,房屋租赁合同纠纷,民事案件,上海市闵行区人民法院,"汪秀维,毛明华,汪正奎,汪小娟",2017-09-07,NaN,2014-11-10,上海市闵行区人民法院\n民 事 裁 定 书\n（2014）闵民五(民)初字第2485号\n原...,一审,（2014）闵民五（民）初字第2485号


In [6]:
my_data_test = df[1:1000]

In [8]:
print(len(set(my_data_test['cause_of_action'])))

118


In [24]:
a = np.array([1])

In [ ]:
a.squeeze

In [42]:
s = "                                                                                                                                                                                                                                  "
my_data = my_data_test.iloc[7]
print()

a = tokenizer.encode( str(my_data['party']) + s[0:32-len(my_data['party'])] + str(my_data['name'])+ s[0:64-len(my_data['name'])]+ str(my_data['cause_of_action']+s[0:32-len(my_data['cause_of_action'])] ), return_tensors="pt")


In [43]:
output= model(a)

In [50]:
a = output.pooler_output.detach().numpy().squeeze()

In [51]:
print(len(a))

768


In [53]:
data_index = {}

for i, my_data in tqdm(my_data_test.iterrows()):
    data_index[my_data['id']] = [my_data['name'],  my_data['cause_of_action'],  my_data['type'],  my_data['date2'],  model(tokenizer.encode(str(my_data['name'])+ s[0:32-len(my_data['name'])]+ str(my_data['cause_of_action']), return_tensors="pt")).pooler_output.detach().numpy().squeeze() ]

In [56]:
my_test_data =[]
for i in data_index.keys():
    my_test_data.append(data_index[i])

# 保距离IsoMap变换的结果测试

In [55]:
from sklearn import decomposition,manifold

embedding_result = manifold.Isomap(n_neighbors=10, n_components=3, )



In [57]:
print(len(my_test_data))

999


In [66]:
my_test_data_728d = []
for i in range(999):
    my_test_data_728d.append(my_test_data[i][4])
    





In [69]:
X_transformed = embedding_result.fit_transform(my_test_data_728d)

In [70]:
print(X_transformed)

[[-3.14848304 -3.7933013   8.78148961]
 [-6.76684722  8.03424264  1.22061685]
 [-5.96840544 -0.9957689  -0.84138444]
 ...
 [-4.95195241 -3.04478831 -0.97569444]
 [-1.16327758  7.9408775  -6.29318234]
 [-6.0056124  -5.76657145 -1.3048459 ]]


In [71]:
print(len(X_transformed))

999


In [76]:
print(type(X_transformed) )

<class 'numpy.ndarray'>


In [73]:
maxX = 0
maxY = 0
maxZ = 0
minX = 0
minY = 0
minZ = 0
for value in X_transformed:
    if value[0]<minX:
        minX = value[0]
    if value[0]>maxX:
        maxX = value[0]
    if value[1]<minY:
        minY = value[1]
    if value[1]>maxY:
        maxY = value[1]
    if value[2]<minZ:
        minZ = value[2]
    if value[2]>maxZ:
        maxZ = value[2]

In [74]:
print(maxX,maxY,maxZ,minX,minY,minZ)
# my_test_data[i] = 'name', 'cause_of_action', 'type', 'date2', 'Vector728'

33.67395932658822 18.551059844277056 18.096303725971858 -13.8769884778519 -12.521884420988734 -12.757566502246226


In [100]:
colors = set()
for data in my_test_data:
    colors.add(data[1])

In [102]:
print(colors)
print(len(colors))

{'抚养费纠纷', '网络购物合同纠纷', '土地租赁合同纠纷', '买卖合同纠纷', '委托合同纠纷', '遗嘱继承纠纷', '名誉权纠纷', '服务合同纠纷', '保安服务合同纠纷', '著作权许可使用合同纠纷', '诈骗', '与公司有关的纠纷', '债权人撤销权纠纷', '合同诈骗', '企业承包经营合同纠纷', '盗窃', '提供劳务者受害责任纠纷', '走私、贩卖、运输、制造毒品', '侵害商业秘密纠纷', '侵害商标权纠纷', '教育机构责任纠纷', '股权转让纠纷', '不当得利纠纷', '抵押权纠纷', '房屋买卖合同纠纷', '合同纠纷', '定作合同纠纷', '挂靠经营合同纠纷', '销售假冒注册商标的商品', '车辆租赁合同纠纷', '民间借贷纠纷', '建筑设备租赁合同纠纷', '寻衅滋事', '故意伤害', '申请确认人民调解协议效力', '离婚纠纷', '房屋租赁合同纠纷', '所有权确认纠纷', '妨害公务', '相邻用水、排水纠纷', '聚众斗殴', '追偿权纠纷', '劳务合同纠纷', '建设工程施工合同纠纷', '公路货物运输合同纠纷', '信用卡纠纷', '申请宣告公民无民事行为能力，申请宣告公民无民事行为能力', '承揽合同纠纷', '医疗损害责任纠纷', '股东知情权纠纷', '非法行医', '饲养动物损害责任纠纷', '著作权权属、侵权纠纷', '经济补偿金纠纷', '电信服务合同纠纷', '物业服务合同纠纷', '保证合同纠纷，保证合同纠纷', '财产损害赔偿纠纷', '融资租赁合同纠纷', '业主知情权纠纷', '公司盈余分配纠纷', '申请公示催告', '非法持有毒品', '共有物分割纠纷', '敲诈勒索', '建设工程合同纠纷', '企业借贷纠纷', '旅店服务合同纠纷', '票据追索权纠纷', '请求变更公司登记纠纷', '信用卡诈骗', '被继承人债务清偿纠纷', '确认合同无效纠纷', '伪造公司、企业、事业单位、人民团体印章', '非法拘禁', '变更抚养关系纠纷', '离婚后财产纠纷', '修理合同纠纷', '装饰装修合同纠纷', '劳动合同纠纷', '侵害作品信息网络传播权纠纷', '相邻关系纠纷', '金融借款合同纠纷，金融借款合同纠纷', '债务转移合同纠纷', '生命权、健康权、身体权纠纷', 

In [103]:
color_map = {}
count = 2
for color in colors:
    count += 2
    color_map[color] = count

In [104]:
print(color_map)

{'抚养费纠纷': 4, '网络购物合同纠纷': 6, '土地租赁合同纠纷': 8, '买卖合同纠纷': 10, '委托合同纠纷': 12, '遗嘱继承纠纷': 14, '名誉权纠纷': 16, '服务合同纠纷': 18, '保安服务合同纠纷': 20, '著作权许可使用合同纠纷': 22, '诈骗': 24, '与公司有关的纠纷': 26, '债权人撤销权纠纷': 28, '合同诈骗': 30, '企业承包经营合同纠纷': 32, '盗窃': 34, '提供劳务者受害责任纠纷': 36, '走私、贩卖、运输、制造毒品': 38, '侵害商业秘密纠纷': 40, '侵害商标权纠纷': 42, '教育机构责任纠纷': 44, '股权转让纠纷': 46, '不当得利纠纷': 48, '抵押权纠纷': 50, '房屋买卖合同纠纷': 52, '合同纠纷': 54, '定作合同纠纷': 56, '挂靠经营合同纠纷': 58, '销售假冒注册商标的商品': 60, '车辆租赁合同纠纷': 62, '民间借贷纠纷': 64, '建筑设备租赁合同纠纷': 66, '寻衅滋事': 68, '故意伤害': 70, '申请确认人民调解协议效力': 72, '离婚纠纷': 74, '房屋租赁合同纠纷': 76, '所有权确认纠纷': 78, '妨害公务': 80, '相邻用水、排水纠纷': 82, '聚众斗殴': 84, '追偿权纠纷': 86, '劳务合同纠纷': 88, '建设工程施工合同纠纷': 90, '公路货物运输合同纠纷': 92, '信用卡纠纷': 94, '申请宣告公民无民事行为能力，申请宣告公民无民事行为能力': 96, '承揽合同纠纷': 98, '医疗损害责任纠纷': 100, '股东知情权纠纷': 102, '非法行医': 104, '饲养动物损害责任纠纷': 106, '著作权权属、侵权纠纷': 108, '经济补偿金纠纷': 110, '电信服务合同纠纷': 112, '物业服务合同纠纷': 114, '保证合同纠纷，保证合同纠纷': 116, '财产损害赔偿纠纷': 118, '融资租赁合同纠纷': 120, '业主知情权纠纷': 122, '公司盈余分配纠纷': 124, '申请公示催告': 126, '非法持有毒品': 128, '共有物分割纠纷': 1

In [105]:
dates = []
for data in my_test_data:
    dates.append( float(data[3][0:4]) + 0.01* float(data[3][5:7]) + 0.0001*float(data[3][8:10]) )

In [108]:
# print(dates)
print(max(dates))
print(min(dates))

2018.0619
2013.0529


In [116]:
print( my_test_data[0] )
content = []
for data in my_test_data:
    content.append( [data[2]+":"+ data[0], color_map[data[1]], (float(data[3][0:4]) + 0.01* float(data[3][5:7]) + 0.0001*float(data[3][8:10])-2012.0529)] )

['深圳市赢时通汽车服务有限公司上海分公司与姜华车辆租赁合同纠纷一审民事裁定书', '车辆租赁合同纠纷', '民事案件', '2017-03-01', array([ 9.15119201e-02,  2.01529533e-01,  7.32176781e-01, -2.59618312e-01,
        4.70229179e-01,  4.18421216e-02, -6.44376457e-01, -5.22643566e-01,
        1.00388601e-01,  4.79966044e-01,  5.90607285e-01,  3.00664663e-01,
       -3.06499958e-01, -4.35020983e-01, -9.76262093e-02,  1.14128955e-01,
       -2.99856693e-01,  2.83816963e-01,  1.62737712e-01, -2.62233093e-02,
        3.63343507e-01,  7.47808302e-03, -3.88346523e-01,  5.10799766e-01,
        1.00879267e-01, -1.26919448e-01,  3.01827312e-01, -1.09115161e-01,
        4.33463991e-01,  6.73740590e-03,  4.92516071e-01, -5.67818284e-01,
        3.29393715e-01,  1.05920658e-01, -4.19076115e-01, -3.59427869e-01,
       -4.85048443e-01,  7.27549434e-01,  3.59344661e-01, -1.78249419e-01,
       -2.63853639e-01, -1.66525751e-01,  7.35434055e-01,  1.43333703e-01,
        1.20170191e-01,  2.01644331e-01,  1.17611565e-01,  2.93879151e-01,
        2.73477435e-01, 

In [117]:
for data in content:
    print(data)

['民事案件:深圳市赢时通汽车服务有限公司上海分公司与姜华车辆租赁合同纠纷一审民事裁定书', 62, 4.977200000000039]
['民事案件:一汽汽车金融有限公司与王以新金融借款合同纠纷一审民事判决书', 204, 5.06009999999992]
['民事案件:上海宝隆物业管理有限公司物业服务合同纠纷一审民事裁定书', 114, 5.049099999999953]
['民事案件:汪秀维、毛明华房屋租赁合同纠纷一审民事裁定书', 76, 2.0580999999999676]
['民事案件:姚美娟与丁晔、丁狄康等共有物分割纠纷一审民事裁定书', 130, 2.038099999999986]
['2:上海银行股份有限公司福民支行与上海恒通经济发展(集团)有限公司、中国音像与数字出版协会等股东损害公司债权人利益责任纠纷一审民事判决书', 180, 2.0195000000001073]
['民事案件:上海邦邦时装有限公司与上海唯煌纺织品有限公司、上海缘品时装有限公司等买卖合同纠纷一审民事裁定书', 10, 2.957600000000184]
['民事案件:上海伟佳物业管理有限公司与杨峰物业服务合同纠纷一审民事裁定书', 114, 2.0191999999999553]
['2:上海盈嘉物业管理有限公司与周婵琼物业服务合同纠纷一审民事裁定书', 114, 3.9799000000000433]
['民事案件:上海峰亿物业管理有限公司与李拓物业服务合同纠纷一审民事判决书', 114, 5.0090999999999894]
['2:李艳红与上海一鼎电子科技有限公司劳动合同纠纷一审民事裁定书', 162, 2.027900000000045]
['民事案件:嘉兴富悦大酒店管理有限公司与上海凯恩耐思服饰有限公司服务合同纠纷一审民事裁定书', 18, 4.048199999999952]
['2:上海静安新成物业有限公司与贺超物业服务合同纠纷案一审民事判决书', 114, 1.047900000000027]
['民事案件:沈红兰、李浩与张波房屋租赁合同纠纷一审民事裁定书', 76, 2.008199999999988]
['刑事案件:郑国良盗窃一审刑事判决书', 34, 5.049800000000005]
['民事案件:上海中星集团申城物业有限公司物业服务

In [72]:
import pyecharts.options as opts
from pyecharts.charts import Scatter3D
import math


# 可视化配置

In [77]:
x_data = [d[0] for d in X_transformed]
y_data = [d[1] for d in X_transformed]
z_data = [d[2] for d in X_transformed]

In [78]:
content_0 = [d[0] for d in my_test_data]
content_1 = [d[0] for d in my_test_data]
content_2 = [d[0] for d in my_test_data]
content_3 = [d[0] for d in my_test_data]
content_4 = [d[0] for d in my_test_data]

In [118]:
my_width =  "1440px"   # 图表的长宽
my_height = "720px"
my_chart_id = None 
my_x_data = x_data  # 坐标轴数据
my_y_data = y_data
my_z_data = z_data  
my_x_axis_name = 'Isomap_Vector_X'  # 坐标轴名称
my_y_axis_name = 'Isomap_Vector_Y'
my_z_axis_name = 'Isomap_Vector_Z'
Grid_x = 75 # 网格大小
Grid_y = 75
Grid_z = 75
# other content and influence size and color need be added to data corrspond dimension
my_content = content  # 需要保证前三列分别是结点需要显示的内容，结点颜色信息，结点尺寸信息，和data轴对齐

In [121]:
# set(my_data_test['cause_of_action']
scatter3D = Scatter3D(
    init_opts = opts.InitOpts(
        width = my_width, 
        height = my_height, 
        chart_id = None,
    )
)

# 设置坐标轴数据
scatter3D.add(
    series_name = "",
    data = [
        [   my_x_data[index],
            my_y_data[index],
            my_z_data[index],    
            index,
            my_content[index][0],
            my_content[index][1],
            my_content[index][2],
        ]
        for index, item in enumerate(my_content)
    ],
    
    xaxis3d_opts=opts.Axis3DOpts(
            name=my_x_axis_name,
            type_="value",
            # textstyle_opts=opts.TextStyleOpts(color="#fff"),
    ),
    yaxis3d_opts=opts.Axis3DOpts(
            name=my_y_axis_name,
            type_="value",
            # textstyle_opts=opts.TextStyleOpts(color="#fff"),
    ),
    zaxis3d_opts=opts.Axis3DOpts(
            name=my_z_axis_name,
            type_="value",
            # textstyle_opts=opts.TextStyleOpts(color="#fff"),
    ),
    grid3d_opts=opts.Grid3DOpts(width = Grid_x, height = Grid_y, depth = Grid_z),

)    
  
# 设置图表表现
scatter3D.set_global_opts(
    visualmap_opts = [
        opts.VisualMapOpts(
            type_ = "color",
            is_calculable = True,
            dimension=5,
            pos_top = "10",
            max_ = 255,
            range_color=[
                    "#1710c0",
                    "#0b9df0",
                    "#00fea8",
                    "#00ff0d",
                    "#f5f811",
                    "#f09a09",
                    "#fe0300",
                ],
        ),
        opts.VisualMapOpts(
            type_ = "size",
            dimension=6,
            pos_bottom="10",
            max_ = 6.0,
            range_size=[10,40],    
        )
    ]
)

scatter3D.set_serites
    

In [122]:
scatter3D.render("my_results1.html")

'/home/zhouyifan/milvus/test/my_results1.html'

In [123]:
print(my_test_data[0][4])

[ 9.15119201e-02  2.01529533e-01  7.32176781e-01 -2.59618312e-01
  4.70229179e-01  4.18421216e-02 -6.44376457e-01 -5.22643566e-01
  1.00388601e-01  4.79966044e-01  5.90607285e-01  3.00664663e-01
 -3.06499958e-01 -4.35020983e-01 -9.76262093e-02  1.14128955e-01
 -2.99856693e-01  2.83816963e-01  1.62737712e-01 -2.62233093e-02
  3.63343507e-01  7.47808302e-03 -3.88346523e-01  5.10799766e-01
  1.00879267e-01 -1.26919448e-01  3.01827312e-01 -1.09115161e-01
  4.33463991e-01  6.73740590e-03  4.92516071e-01 -5.67818284e-01
  3.29393715e-01  1.05920658e-01 -4.19076115e-01 -3.59427869e-01
 -4.85048443e-01  7.27549434e-01  3.59344661e-01 -1.78249419e-01
 -2.63853639e-01 -1.66525751e-01  7.35434055e-01  1.43333703e-01
  1.20170191e-01  2.01644331e-01  1.17611565e-01  2.93879151e-01
  2.73477435e-01 -5.75407267e-01 -5.81927299e-01  5.15113771e-01
 -2.06837505e-01 -1.49293557e-01  4.27483618e-01 -4.74285632e-02
 -4.84054923e-01 -1.10240027e-01 -4.88010794e-01 -1.90010220e-01
  5.57541132e-01  3.72292

In [124]:
print(len(my_test_data[0][4]))

768


In [125]:
print(len(X_transformed))

999


In [126]:
print(len(my_test_data))

999


In [131]:
my_test_data_new = []
for i in data_index.keys():
    my_test_data_new.append([i,data_index[i]])

In [132]:
print(my_test_data_new[87])

[1295097, ['上海金榜物业有限公司与郭仁官物业服务合同纠纷一审民事裁定书', '物业服务合同纠纷', '民事案件', '2015-07-10', array([-1.18703485e-01,  9.49576050e-02,  7.94341266e-01, -5.76845944e-01,
        5.12404680e-01, -2.10466087e-01, -6.21162057e-01, -6.19794965e-01,
        1.42202020e-01,  4.83566254e-01,  6.35912299e-01,  3.72516215e-01,
       -4.44092214e-01, -2.16894686e-01, -3.30044031e-01, -1.88803121e-01,
       -4.76041555e-01,  3.14285792e-02,  2.37379633e-02, -1.36640683e-01,
        5.80443799e-01,  3.34983766e-01, -3.04740518e-01,  4.43539917e-01,
        1.46932989e-01, -5.78777969e-01,  6.80952847e-01, -3.34865719e-01,
       -2.94656754e-02,  7.52328038e-02,  5.91037750e-01, -4.88465160e-01,
        1.61530197e-01,  5.27503863e-02, -3.29635322e-01, -3.98334444e-01,
       -3.88096064e-01,  6.25743032e-01,  2.63362914e-01,  2.26215318e-01,
       -1.19102247e-01, -2.34887555e-01,  7.54581869e-01,  1.44236773e-01,
        1.25917912e-01, -4.78032790e-02, -1.13907829e-01,  3.80738676e-01,
        3.80242616e-01

In [133]:
for i in range(len(my_test_data_new)):
    my_test_data_new[i].append(X_transformed[i])

In [134]:
print(my_test_data_new[50])

[1295052, ['金奎、王林芳与范爱军民间借贷纠纷一审民事判决书', '民间借贷纠纷', '民事案件', '2017-07-10', array([-0.02099071,  0.1234716 ,  0.52283037, -0.4138812 ,  0.0489493 ,
       -0.29878518, -0.7177629 , -0.64729834,  0.29840255,  0.4398556 ,
        0.45012167,  0.34857914, -0.5644899 , -0.36725548,  0.06522607,
       -0.09065665, -0.54118556,  0.355007  ,  0.2438675 , -0.34519   ,
        0.66014534,  0.1710501 , -0.07512225,  0.47979143,  0.3354861 ,
       -0.34047887,  0.6750913 , -0.33589357,  0.29459453,  0.15842716,
        0.26802698, -0.66531587,  0.10001311,  0.250894  , -0.1675382 ,
        0.02143162,  0.00459601,  0.3025254 ,  0.15097845,  0.14179775,
        0.0115921 , -0.250653  ,  0.55578893,  0.52803445,  0.24451096,
       -0.14064725, -0.1593009 ,  0.12744024,  0.1928972 , -0.53468287,
       -0.6019242 ,  0.24937981,  0.0664519 ,  0.04664455,  0.3464888 ,
       -0.24349819, -0.01703374, -0.42248514, -0.25885767,  0.03605039,
        0.48575583, -0.00443492,  0.4483744 ,  0.09282823, -0.0354

In [143]:
test_name = []
test_id = []
test_vector = []
test_vector_transformed = [] 
for i in range(1,11):
    test_name.append(my_test_data_new[int(i ** 1.5 + 23 % i + 42//i)][1][0])
    test_id.append(my_test_data_new[int(i ** 1.5 + 23 % i + 42//i)][0])
    test_vector.append(my_test_data_new[int(i ** 1.5 + 23 % i + 42//i)][1][4])
    test_vector_transformed.append(my_test_data_new[int(i ** 1.5 + 23 % i + 42//i)][2])

# 随机采样测试保距变换欧式距离的变换，事实证明似乎还不存在欧式距离相对大小不变的降维。ISO只保持了结点路径距离的不变，符合流行假设下的保距变换，但是高维空间中的欧式距离不可用，而向量化检索一般使用L2测度或余弦相似度测度

# 保距变换和保角变换均不存在，也不存在低维空间的直接编码

In [144]:
print(test_name)
print(test_id)
print(test_vector)
print(test_vector_transformed)

['原告上海鼎高物业管理有限公司与被告上海市闵行区都市宜家小区业主大会企业借贷纠纷一审民事判决书', '上海品哲配线器材有限公司与上海龙能自动化设备有限公司买卖合同纠纷一审民事裁定书', '张庆霞与阳光财产保险股份有限公司上海市分公司财产保险合同纠纷一审民事裁定书', '张庆霞与阳光财产保险股份有限公司上海市分公司财产保险合同纠纷一审民事裁定书', '张颖与上海梦勰信息技术有限公司劳动合同纠纷一审民事裁定书', '吴鸣与大众汽车租赁有限公司、朱鸣其他合同纠纷一审民事裁定书', '吴鸣与大众汽车租赁有限公司、朱鸣其他合同纠纷一审民事裁定书', '袁毅超与顾伟民间借贷纠纷一审民事判决书', '杨某1、张群丽等与唐雨房屋租赁合同纠纷一审民事裁定书', '袁国富与潘益清、董连忠民间借贷纠纷一审民事判决书']
[1295044, 1295021, 1295018, 1295018, 1295019, 1295023, 1295023, 1295032, 1295035, 1295038]
[array([-2.22121954e-01,  2.70647943e-01,  8.14302802e-01, -5.32778084e-01,
        5.52655578e-01, -1.56291798e-01, -3.84977669e-01, -7.79555261e-01,
        2.76056677e-01,  4.26933676e-01,  6.77738965e-01,  4.01446193e-01,
       -5.09674549e-01, -3.78341764e-01, -3.39075208e-01, -2.82630533e-01,
       -4.41631019e-01, -2.43703779e-02, -5.83964661e-02, -1.55707672e-01,
        6.44185960e-01,  1.29030228e-01, -2.58632481e-01,  4.86759067e-01,
        2.26471856e-01, -5.30741274e-01,  4.52491552e-01, -5.06140172e-01,
        1.70195065e-02,  1.7

In [145]:
import numpy as np

In [158]:
test_vector_rank = []
for i in range(0,10):
    test_vector_rank_i = [] 
    for j in range(0,10):
        test_vector_rank_i.append( (test_name[j],test_id[j], np.linalg.norm( test_vector[i]-test_vector[j] ))) 
    test_vector_rank.append(sorted(test_vector_rank_i,key = lambda x: x[2]))

In [159]:
from pprint import pprint
pprint(test_vector_rank[0])

[('原告上海鼎高物业管理有限公司与被告上海市闵行区都市宜家小区业主大会企业借贷纠纷一审民事判决书', 1295044, 0.0),
 ('上海品哲配线器材有限公司与上海龙能自动化设备有限公司买卖合同纠纷一审民事裁定书', 1295021, 5.2570863),
 ('张庆霞与阳光财产保险股份有限公司上海市分公司财产保险合同纠纷一审民事裁定书', 1295018, 5.56462),
 ('张庆霞与阳光财产保险股份有限公司上海市分公司财产保险合同纠纷一审民事裁定书', 1295018, 5.56462),
 ('张颖与上海梦勰信息技术有限公司劳动合同纠纷一审民事裁定书', 1295019, 5.6141777),
 ('袁国富与潘益清、董连忠民间借贷纠纷一审民事判决书', 1295038, 6.46476),
 ('袁毅超与顾伟民间借贷纠纷一审民事判决书', 1295032, 6.5799212),
 ('杨某1、张群丽等与唐雨房屋租赁合同纠纷一审民事裁定书', 1295035, 6.601683),
 ('吴鸣与大众汽车租赁有限公司、朱鸣其他合同纠纷一审民事裁定书', 1295023, 6.72516),
 ('吴鸣与大众汽车租赁有限公司、朱鸣其他合同纠纷一审民事裁定书', 1295023, 6.72516)]


In [160]:
test_vector_transformed_rank = []
for i in range(0,10):
    test_vector_rank_i = [] 
    for j in range(0,10):
        test_vector_rank_i.append( (test_name[j],test_id[j], np.linalg.norm( test_vector_transformed[i]-test_vector_transformed[j] )))
    test_vector_transformed_rank.append(sorted(test_vector_rank_i,key = lambda x: x[2]))

In [161]:
pprint(test_vector_transformed_rank[0])

[('原告上海鼎高物业管理有限公司与被告上海市闵行区都市宜家小区业主大会企业借贷纠纷一审民事判决书', 1295044, 0.0),
 ('张颖与上海梦勰信息技术有限公司劳动合同纠纷一审民事裁定书', 1295019, 6.694524091189509),
 ('上海品哲配线器材有限公司与上海龙能自动化设备有限公司买卖合同纠纷一审民事裁定书', 1295021, 8.223232850085905),
 ('袁毅超与顾伟民间借贷纠纷一审民事判决书', 1295032, 15.015262140638514),
 ('吴鸣与大众汽车租赁有限公司、朱鸣其他合同纠纷一审民事裁定书', 1295023, 15.229703334230697),
 ('吴鸣与大众汽车租赁有限公司、朱鸣其他合同纠纷一审民事裁定书', 1295023, 15.229703334230697),
 ('杨某1、张群丽等与唐雨房屋租赁合同纠纷一审民事裁定书', 1295035, 15.468072975063574),
 ('袁国富与潘益清、董连忠民间借贷纠纷一审民事判决书', 1295038, 16.501461156845274),
 ('张庆霞与阳光财产保险股份有限公司上海市分公司财产保险合同纠纷一审民事裁定书', 1295018, 18.651422404551308),
 ('张庆霞与阳光财产保险股份有限公司上海市分公司财产保险合同纠纷一审民事裁定书', 1295018, 18.651422404551308)]


In [162]:
pprint(test_vector_rank[4])
pprint(test_vector_transformed_rank[4])

[('张颖与上海梦勰信息技术有限公司劳动合同纠纷一审民事裁定书', 1295019, 0.0),
 ('上海品哲配线器材有限公司与上海龙能自动化设备有限公司买卖合同纠纷一审民事裁定书', 1295021, 4.84576),
 ('张庆霞与阳光财产保险股份有限公司上海市分公司财产保险合同纠纷一审民事裁定书', 1295018, 5.196673),
 ('张庆霞与阳光财产保险股份有限公司上海市分公司财产保险合同纠纷一审民事裁定书', 1295018, 5.196673),
 ('吴鸣与大众汽车租赁有限公司、朱鸣其他合同纠纷一审民事裁定书', 1295023, 5.498497),
 ('吴鸣与大众汽车租赁有限公司、朱鸣其他合同纠纷一审民事裁定书', 1295023, 5.498497),
 ('原告上海鼎高物业管理有限公司与被告上海市闵行区都市宜家小区业主大会企业借贷纠纷一审民事判决书', 1295044, 5.6141777),
 ('杨某1、张群丽等与唐雨房屋租赁合同纠纷一审民事裁定书', 1295035, 5.628419),
 ('袁毅超与顾伟民间借贷纠纷一审民事判决书', 1295032, 5.871521),
 ('袁国富与潘益清、董连忠民间借贷纠纷一审民事判决书', 1295038, 5.986354)]
[('张颖与上海梦勰信息技术有限公司劳动合同纠纷一审民事裁定书', 1295019, 0.0),
 ('上海品哲配线器材有限公司与上海龙能自动化设备有限公司买卖合同纠纷一审民事裁定书', 1295021, 6.230309079517489),
 ('原告上海鼎高物业管理有限公司与被告上海市闵行区都市宜家小区业主大会企业借贷纠纷一审民事判决书', 1295044, 6.694524091189509),
 ('袁毅超与顾伟民间借贷纠纷一审民事判决书', 1295032, 13.57816032497732),
 ('吴鸣与大众汽车租赁有限公司、朱鸣其他合同纠纷一审民事裁定书', 1295023, 13.62387872417752),
 ('吴鸣与大众汽车租赁有限公司、朱鸣其他合同纠纷一审民事裁定书', 1295023, 13.62387872417752),
 ('袁国富与潘益清、董连忠民间借贷纠纷一审民事判决书', 1295038, 13.973

In [164]:
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=3, init = "pca", )

In [166]:
tsne_results = X_embedded.fit_transform(my_test_data_728d)

In [167]:
print(tsne_results.shape)

(999, 3)


In [177]:
maxX = 0
maxY = 0
maxZ = 0
minX = 0
minY = 0
minZ = 0
for value in tsne_results:
    if value[0]<minX:
        minX = value[0]
    if value[0]>maxX:
        maxX = value[0]
    if value[1]<minY:
        minY = value[1]
    if value[1]>maxY:
        maxY = value[1]
    if value[2]<minZ:
        minZ = value[2]
    if value[2]>maxZ:
        maxZ = value[2]

In [178]:
print(maxX,maxY,maxZ,minX,minY,minZ)

56.195904 55.824703 49.81878 -126.494934 -49.758686 -54.39277


In [196]:
x_tsne_data = [ (i[0]-minX)/(maxX-minX)*50-25 for i in tsne_results]
y_tsne_data = [ (i[1]-minY)/(maxY-minY)*50-25 for i in tsne_results]
z_tsne_data = [ (i[2]-minZ)/(maxZ-minZ)*50-25 for i in tsne_results]

In [197]:
my_width =  "1440px"   # 图表的长宽
my_height = "720px"
my_chart_id = None 
my_x_data = x_tsne_data  # 坐标轴数据
my_y_data = y_tsne_data
my_z_data = z_tsne_data 
my_x_axis_name = 'tsnemap_Vector_X'  # 坐标轴名称
my_y_axis_name = 'tsnemap_Vector_Y'
my_z_axis_name = 'tsnemap_Vector_Z'
Grid_x = 100 # 网格大小
Grid_y = 100
Grid_z = 100
# other content and influence size and color need be added to data corrspond dimension
my_content = content  # 需要保证前三列分别是结点需要显示的内容，结点颜色信息，结点尺寸信息，和data轴对齐

In [198]:
# set(my_data_test['cause_of_action']
scatter3D = Scatter3D(
    init_opts = opts.InitOpts(
        width = my_width, 
        height = my_height, 
        chart_id = None,
    )
)

# 设置坐标轴数据
scatter3D.add(
    series_name = "",
    data = [
        [   my_x_data[index],
            my_y_data[index],
            my_z_data[index],    
            index,
            my_content[index][0],
            my_content[index][1],
            my_content[index][2],
        ]
        for index, item in enumerate(my_content)
    ],
    
    xaxis3d_opts=opts.Axis3DOpts(
            name=my_x_axis_name,
            type_="value",
            # textstyle_opts=opts.TextStyleOpts(color="#fff"),
    ),
    yaxis3d_opts=opts.Axis3DOpts(
            name=my_y_axis_name,
            type_="value",
            # textstyle_opts=opts.TextStyleOpts(color="#fff"),
    ),
    zaxis3d_opts=opts.Axis3DOpts(
            name=my_z_axis_name,
            type_="value",
            # textstyle_opts=opts.TextStyleOpts(color="#fff"),
    ),
    grid3d_opts=opts.Grid3DOpts(width = Grid_x, height = Grid_y, depth = Grid_z),

)    
  
# 设置图表表现
scatter3D.set_global_opts(
    visualmap_opts = [
        opts.VisualMapOpts(
            type_ = "color",
            is_calculable = True,
            dimension=5,
            pos_top = "10",
            max_ = 255,
            range_color=[
                    "#1710c0",
                    "#0b9df0",
                    "#00fea8",
                    "#00ff0d",
                    "#f5f811",
                    "#f09a09",
                    "#fe0300",
                ],
        ),
        opts.VisualMapOpts(
            type_ = "size",
            dimension=6,
            pos_bottom="10",
            max_ = 6.0,
            range_size=[10,40],    
        )
    ]
)

#scatter3D.set_serites

In [199]:
scatter3D.render("my_results2.html")

'/home/zhouyifan/milvus/test/my_results2.html'

# 将728维向量build进入Milvus当中，可以通过检索类似案件Milvus+Lawformer模型检索

In [223]:
from milvus import Milvus, IndexType, MetricType, Status
__HOST = '127.0.0.1'
__PORT = '19530'
# __PORT = '19121'

__DIM = 768
__INDEX_FILE_SIZE = 1024 # max file stored


def milvus_build(name, metric_type=None):
    """
    input parameter:
    name: Wanting Create Collection Name, data vector space name
    metric_type: way to compute similiarity
    output parameter:
    instance of our milvus server
    """
    milvus = Milvus(__HOST, __PORT)

    collection_name = name

    # create a vector space named collection_name
    status, ok = milvus.has_collection(collection_name)
    if not ok:
        param = {
                'collection_name': collection_name,
                'dimension': __DIM,
                'index_file_size': __INDEX_FILE_SIZE,  # optional
                'metric_type': MetricType.L2 if metric_type == None else metric_type  # optional
        }
    
        milvus.create_collection(param)
        print("Create "+name+" Successfully")

    _, collections = milvus.list_collections()
    print("Now our collections list: \n", [content + '\n' for content in collections])

    return milvus




# 'MinHang_Law_Vector_Test'

In [224]:
space_name = "MinHang_Law_Vector_Test_768"
milvus = milvus_build(space_name)

Create MinHang_Law_Vector_Test_768 Successfully
Now our collections list: 
 ['example_collection_\n', 'three_dimension_demo\n', 'MinHang_Law_Vector_Test\n', 'MinHang_Law_Vector_Test_728\n', 'MinHang_Law_Vector_Test_768\n']


In [225]:
vectors_milvus = []
for i in my_test_data_728d:
    vectors_milvus.append(list(i))

In [226]:
print(len(vectors_milvus))
print(len(vectors_milvus[0]))

999
768


In [227]:
status, ids = milvus.insert(collection_name=space_name, records= vectors_milvus )
if not status.OK():
        print("Insert failed: ".format(status))

In [228]:
print(ids)

[1668608354314194000, 1668608354314194001, 1668608354314194002, 1668608354314194003, 1668608354314194004, 1668608354314194005, 1668608354314194006, 1668608354314194007, 1668608354314194008, 1668608354314194009, 1668608354314194010, 1668608354314194011, 1668608354314194012, 1668608354314194013, 1668608354314194014, 1668608354314194015, 1668608354314194016, 1668608354314194017, 1668608354314194018, 1668608354314194019, 1668608354314194020, 1668608354314194021, 1668608354314194022, 1668608354314194023, 1668608354314194024, 1668608354314194025, 1668608354314194026, 1668608354314194027, 1668608354314194028, 1668608354314194029, 1668608354314194030, 1668608354314194031, 1668608354314194032, 1668608354314194033, 1668608354314194034, 1668608354314194035, 1668608354314194036, 1668608354314194037, 1668608354314194038, 1668608354314194039, 1668608354314194040, 1668608354314194041, 1668608354314194042, 1668608354314194043, 1668608354314194044, 1668608354314194045, 1668608354314194046, 166860835431

In [229]:
for i in range(len(my_test_data_new)):
    my_test_data_new[i].append(ids[i])

In [231]:
print(my_test_data_new[0])

[1294989, ['深圳市赢时通汽车服务有限公司上海分公司与姜华车辆租赁合同纠纷一审民事裁定书', '车辆租赁合同纠纷', '民事案件', '2017-03-01', array([ 9.15119201e-02,  2.01529533e-01,  7.32176781e-01, -2.59618312e-01,
        4.70229179e-01,  4.18421216e-02, -6.44376457e-01, -5.22643566e-01,
        1.00388601e-01,  4.79966044e-01,  5.90607285e-01,  3.00664663e-01,
       -3.06499958e-01, -4.35020983e-01, -9.76262093e-02,  1.14128955e-01,
       -2.99856693e-01,  2.83816963e-01,  1.62737712e-01, -2.62233093e-02,
        3.63343507e-01,  7.47808302e-03, -3.88346523e-01,  5.10799766e-01,
        1.00879267e-01, -1.26919448e-01,  3.01827312e-01, -1.09115161e-01,
        4.33463991e-01,  6.73740590e-03,  4.92516071e-01, -5.67818284e-01,
        3.29393715e-01,  1.05920658e-01, -4.19076115e-01, -3.59427869e-01,
       -4.85048443e-01,  7.27549434e-01,  3.59344661e-01, -1.78249419e-01,
       -2.63853639e-01, -1.66525751e-01,  7.35434055e-01,  1.43333703e-01,
        1.20170191e-01,  2.01644331e-01,  1.17611565e-01,  2.93879151e-01,
        2.7347

In [232]:
my_test_data_new_web_return = {}
for data in my_test_data_new:
    my_test_data_new_web_return[data[3]] = data[0]

In [233]:
print(my_test_data_new_web_return)

{1668608354314194000: 1294989, 1668608354314194001: 1294990, 1668608354314194002: 1294993, 1668608354314194003: 1294994, 1668608354314194004: 1294995, 1668608354314194005: 1294996, 1668608354314194006: 1294997, 1668608354314194007: 1294998, 1668608354314194008: 1295000, 1668608354314194009: 1295001, 1668608354314194010: 1295002, 1668608354314194011: 1295003, 1668608354314194012: 1295005, 1668608354314194013: 1295006, 1668608354314194014: 1295008, 1668608354314194015: 1295010, 1668608354314194016: 1295012, 1668608354314194017: 1295013, 1668608354314194018: 1295015, 1668608354314194019: 1295016, 1668608354314194020: 1295017, 1668608354314194021: 1295018, 1668608354314194022: 1295019, 1668608354314194023: 1295020, 1668608354314194024: 1295021, 1668608354314194025: 1295022, 1668608354314194026: 1295023, 1668608354314194027: 1295024, 1668608354314194028: 1295025, 1668608354314194029: 1295026, 1668608354314194030: 1295027, 1668608354314194031: 1295028, 1668608354314194032: 1295029, 166860835

In [236]:
from tqdm import tqdm

my_data_test.head()

,id,name,cause_of_action,type,court,party,date1,procuratorate,date2,doc,instance,num
1,1294989,深圳市赢时通汽车服务有限公司上海分公司与姜华车辆租赁合同纠纷一审民事裁定书,车辆租赁合同纠纷,民事案件,上海市闵行区人民法院,"深圳市赢时通汽车服务有限公司上海分公司,姜华",2018-01-30,NaN,2017-03-01,上海市闵行区人民法院\n民 事 裁 定 书\n（2017）沪0112民初2332号\n原告：...,一审,（2017）沪0112民初2332号
2,1294990,一汽汽车金融有限公司与王以新金融借款合同纠纷一审民事判决书,金融借款合同纠纷,民事案件,上海市闵行区人民法院,"汽汽车金融有限公司,王以新",2018-04-01,NaN,2017-11-30,上海市闵行区人民法院\n民 事 判 决 书\n（2017）沪0112民初15632号\n原告...,一审,（2017）沪0112民初15632号
3,1294993,上海宝隆物业管理有限公司物业服务合同纠纷一审民事裁定书,物业服务合同纠纷,民事案件,上海市闵行区人民法院,"上海宝隆物业管理有限公司,樊美芬",2017-11-03,NaN,2017-10-20,上海市闵行区人民法院\n民 事 裁 定 书\n（2017）沪0112民初33632号\n原告...,一审,（2017）沪0112民初33632号
4,1294994,汪秀维、毛明华房屋租赁合同纠纷一审民事裁定书,房屋租赁合同纠纷,民事案件,上海市闵行区人民法院,"汪秀维,毛明华,汪正奎,汪小娟",2017-09-07,NaN,2014-11-10,上海市闵行区人民法院\n民 事 裁 定 书\n（2014）闵民五(民)初字第2485号\n原...,一审,（2014）闵民五（民）初字第2485号
5,1294995,姚美娟与丁晔、丁狄康等共有物分割纠纷一审民事裁定书,共有物分割纠纷,民事案件,上海市闵行区人民法院,"姚美娟,丁晔,丁狄康,丁泽斌",2017-09-19,NaN,2014-09-10,上海市闵行区人民法院\n民 事 裁 定 书\n（2014）闵民一(民)初字第14435号\n...,一审,（2014）闵民一（民）初字第14435号


In [238]:
data_map = {}
for i, my_data in tqdm(my_data_test.iterrows()):
    data_map[my_data['id']] = i

999it [00:00, 11424.38it/s]


In [239]:
print(data_map)

{1294989: 1, 1294990: 2, 1294993: 3, 1294994: 4, 1294995: 5, 1294996: 6, 1294997: 7, 1294998: 8, 1295000: 9, 1295001: 10, 1295002: 11, 1295003: 12, 1295005: 13, 1295006: 14, 1295008: 15, 1295010: 16, 1295012: 17, 1295013: 18, 1295015: 19, 1295016: 20, 1295017: 21, 1295018: 22, 1295019: 23, 1295020: 24, 1295021: 25, 1295022: 26, 1295023: 27, 1295024: 28, 1295025: 29, 1295026: 30, 1295027: 31, 1295028: 32, 1295029: 33, 1295030: 34, 1295032: 35, 1295033: 36, 1295035: 37, 1295037: 38, 1295038: 39, 1295040: 40, 1295041: 41, 1295042: 42, 1295043: 43, 1295044: 44, 1295046: 45, 1295047: 46, 1295048: 47, 1295049: 48, 1295050: 49, 1295051: 50, 1295052: 51, 1295054: 52, 1295056: 53, 1295057: 54, 1295058: 55, 1295059: 56, 1295060: 57, 1295061: 58, 1295062: 59, 1295063: 60, 1295064: 61, 1295065: 62, 1295066: 63, 1295067: 64, 1295068: 65, 1295069: 66, 1295070: 67, 1295073: 68, 1295074: 69, 1295075: 70, 1295076: 71, 1295077: 72, 1295078: 73, 1295079: 74, 1295080: 75, 1295081: 76, 1295082: 77, 1295083

In [240]:
my_data_test.to_csv('my_test_demo.csv')

In [241]:
jieguo = pd.read_csv('../../Web_Outer_File/my_test_demo.csv')

In [242]:
jieguo.head()

,Unnamed: 0,id,name,cause_of_action,type,court,party,date1,procuratorate,date2,doc,instance,num
0,1,1294989,深圳市赢时通汽车服务有限公司上海分公司与姜华车辆租赁合同纠纷一审民事裁定书,车辆租赁合同纠纷,民事案件,上海市闵行区人民法院,"深圳市赢时通汽车服务有限公司上海分公司,姜华",2018-01-30,NaN,2017-03-01,上海市闵行区人民法院\n民 事 裁 定 书\n（2017）沪0112民初2332号\n原告：...,一审,（2017）沪0112民初2332号
1,2,1294990,一汽汽车金融有限公司与王以新金融借款合同纠纷一审民事判决书,金融借款合同纠纷,民事案件,上海市闵行区人民法院,"汽汽车金融有限公司,王以新",2018-04-01,NaN,2017-11-30,上海市闵行区人民法院\n民 事 判 决 书\n（2017）沪0112民初15632号\n原告...,一审,（2017）沪0112民初15632号
2,3,1294993,上海宝隆物业管理有限公司物业服务合同纠纷一审民事裁定书,物业服务合同纠纷,民事案件,上海市闵行区人民法院,"上海宝隆物业管理有限公司,樊美芬",2017-11-03,NaN,2017-10-20,上海市闵行区人民法院\n民 事 裁 定 书\n（2017）沪0112民初33632号\n原告...,一审,（2017）沪0112民初33632号
3,4,1294994,汪秀维、毛明华房屋租赁合同纠纷一审民事裁定书,房屋租赁合同纠纷,民事案件,上海市闵行区人民法院,"汪秀维,毛明华,汪正奎,汪小娟",2017-09-07,NaN,2014-11-10,上海市闵行区人民法院\n民 事 裁 定 书\n（2014）闵民五(民)初字第2485号\n原...,一审,（2014）闵民五（民）初字第2485号
4,5,1294995,姚美娟与丁晔、丁狄康等共有物分割纠纷一审民事裁定书,共有物分割纠纷,民事案件,上海市闵行区人民法院,"姚美娟,丁晔,丁狄康,丁泽斌",2017-09-19,NaN,2014-09-10,上海市闵行区人民法院\n民 事 裁 定 书\n（2014）闵民一(民)初字第14435号\n...,一审,（2014）闵民一（民）初字第14435号


In [243]:
print('ids')

ids


In [244]:
print(ids)

[1668608354314194000, 1668608354314194001, 1668608354314194002, 1668608354314194003, 1668608354314194004, 1668608354314194005, 1668608354314194006, 1668608354314194007, 1668608354314194008, 1668608354314194009, 1668608354314194010, 1668608354314194011, 1668608354314194012, 1668608354314194013, 1668608354314194014, 1668608354314194015, 1668608354314194016, 1668608354314194017, 1668608354314194018, 1668608354314194019, 1668608354314194020, 1668608354314194021, 1668608354314194022, 1668608354314194023, 1668608354314194024, 1668608354314194025, 1668608354314194026, 1668608354314194027, 1668608354314194028, 1668608354314194029, 1668608354314194030, 1668608354314194031, 1668608354314194032, 1668608354314194033, 1668608354314194034, 1668608354314194035, 1668608354314194036, 1668608354314194037, 1668608354314194038, 1668608354314194039, 1668608354314194040, 1668608354314194041, 1668608354314194042, 1668608354314194043, 1668608354314194044, 1668608354314194045, 1668608354314194046, 166860835431

In [247]:
my_test_data_new_web_return[ids[0]]

1294989

In [248]:
data_map[1294989]

1

In [249]:
my_test_data[1]

['一汽汽车金融有限公司与王以新金融借款合同纠纷一审民事判决书', '金融借款合同纠纷', '民事案件', '2017-11-30', array([-0.04098145, -0.07913876,  0.65610194, -0.05031793,  0.26956388,
        0.09825972, -0.74895704, -0.55995244,  0.11032308,  0.26464203,
        0.5650287 ,  0.2800382 , -0.36460286, -0.10075969, -0.0233793 ,
        0.06526499, -0.42004767,  0.39853442,  0.23293859, -0.2818447 ,
        0.28471315,  0.06811447, -0.25221825,  0.64591545,  0.30725816,
       -0.21929836,  0.6690412 , -0.27051762,  0.4644263 ,  0.15025754,
        0.2019657 , -0.535035  ,  0.29111537,  0.32734632, -0.46154425,
       -0.22691314, -0.26766977,  0.60335726,  0.10899808, -0.13801011,
       -0.13842845, -0.11744532,  0.5069298 ,  0.3287869 ,  0.3167926 ,
       -0.03540545, -0.3463578 ,  0.16303341, -0.04740725, -0.60159224,
       -0.733354  ,  0.30346555, -0.01607109,  0.16078109,  0.24947114,
       -0.3213684 , -0.31290635, -0.5707405 , -0.48547018, -0.16547543,
        0.49949935, -0.01524763,  0.41795996,  0.04989539,  0.152028

In [250]:
my_test_data[0]

['深圳市赢时通汽车服务有限公司上海分公司与姜华车辆租赁合同纠纷一审民事裁定书', '车辆租赁合同纠纷', '民事案件', '2017-03-01', array([ 9.15119201e-02,  2.01529533e-01,  7.32176781e-01, -2.59618312e-01,
        4.70229179e-01,  4.18421216e-02, -6.44376457e-01, -5.22643566e-01,
        1.00388601e-01,  4.79966044e-01,  5.90607285e-01,  3.00664663e-01,
       -3.06499958e-01, -4.35020983e-01, -9.76262093e-02,  1.14128955e-01,
       -2.99856693e-01,  2.83816963e-01,  1.62737712e-01, -2.62233093e-02,
        3.63343507e-01,  7.47808302e-03, -3.88346523e-01,  5.10799766e-01,
        1.00879267e-01, -1.26919448e-01,  3.01827312e-01, -1.09115161e-01,
        4.33463991e-01,  6.73740590e-03,  4.92516071e-01, -5.67818284e-01,
        3.29393715e-01,  1.05920658e-01, -4.19076115e-01, -3.59427869e-01,
       -4.85048443e-01,  7.27549434e-01,  3.59344661e-01, -1.78249419e-01,
       -2.63853639e-01, -1.66525751e-01,  7.35434055e-01,  1.43333703e-01,
        1.20170191e-01,  2.01644331e-01,  1.17611565e-01,  2.93879151e-01,
        2.73477435e-01, 

In [252]:
my_test_data[0:10]

[['深圳市赢时通汽车服务有限公司上海分公司与姜华车辆租赁合同纠纷一审民事裁定书', '车辆租赁合同纠纷', '民事案件', '2017-03-01', array([ 9.15119201e-02,  2.01529533e-01,  7.32176781e-01, -2.59618312e-01,
        4.70229179e-01,  4.18421216e-02, -6.44376457e-01, -5.22643566e-01,
        1.00388601e-01,  4.79966044e-01,  5.90607285e-01,  3.00664663e-01,
       -3.06499958e-01, -4.35020983e-01, -9.76262093e-02,  1.14128955e-01,
       -2.99856693e-01,  2.83816963e-01,  1.62737712e-01, -2.62233093e-02,
        3.63343507e-01,  7.47808302e-03, -3.88346523e-01,  5.10799766e-01,
        1.00879267e-01, -1.26919448e-01,  3.01827312e-01, -1.09115161e-01,
        4.33463991e-01,  6.73740590e-03,  4.92516071e-01, -5.67818284e-01,
        3.29393715e-01,  1.05920658e-01, -4.19076115e-01, -3.59427869e-01,
       -4.85048443e-01,  7.27549434e-01,  3.59344661e-01, -1.78249419e-01,
       -2.63853639e-01, -1.66525751e-01,  7.35434055e-01,  1.43333703e-01,
        1.20170191e-01,  2.01644331e-01,  1.17611565e-01,  2.93879151e-01,
        2.73477435e-01,

In [253]:
my_test_data[15:200:20]

[['上海中星集团申城物业有限公司物业服务合同纠纷一审民事裁定书', '物业服务合同纠纷', '民事案件', '2017-06-23', array([-6.29696697e-02,  1.05477497e-01,  7.46549487e-01, -4.04878676e-01,
        6.10642016e-01, -2.73408741e-01, -5.92623830e-01, -5.41421592e-01,
        1.63317576e-01,  4.02280718e-01,  5.53876936e-01,  4.72092330e-01,
       -4.64048326e-01, -3.10136050e-01, -3.32016706e-01, -5.99674992e-02,
       -3.16377044e-01,  7.23211616e-02, -4.62326482e-02, -3.04927468e-01,
        6.60520077e-01,  2.77676046e-01, -3.19169402e-01,  6.01240456e-01,
        1.57977030e-01, -3.21610749e-01,  6.46546006e-01, -3.88516694e-01,
        1.46209514e-02,  1.32485345e-01,  3.09833705e-01, -4.83340114e-01,
        2.66402781e-01,  1.05764076e-01, -1.54725477e-01, -4.38491702e-01,
       -6.18385315e-01,  6.46664798e-01,  4.33555543e-01,  4.19065267e-01,
       -2.06590399e-01, -3.60437065e-01,  7.03203082e-01,  2.39698634e-01,
        2.65020970e-02, -1.24991991e-01,  1.04960360e-01,  3.59997094e-01,
        3.14322680e-01, -4.2187